<!--NAVIGATION-->
< [Results From Parametric Analysis](01.03-Results-From-Parametric-Analysis.ipynb) | [Contents](Index.ipynb) | [Introduction to EnergyPlus Models Scripting](02.00-Introduction-to-EnergyPlus-Models-Scripting.ipynb) >

# Results From Sensitivity Analysis

In [1]:
import os
from surrogate_schema import APIClient

%load_ext dotenv
%dotenv

# typical block to authenticate to the API
client = APIClient(api_url="http://localhost:8080")
client.whoami()  # shows that you are properly authenticated

User(username='s.letellierduchesne', email='sam@cabonsignal.com', full_name='Samuel Letellier-Duchesne', disabled=None, valid_until=datetime.datetime(2025, 1, 29, 15, 28, 26))

In [2]:
from surrogate_schema import BuildingAnalysis

In [3]:
analysis = BuildingAnalysis.parse_file("data/analysis.json")

In [4]:
df = analysis.get_sensitivity_results().to_df()
df

,Variable,WINU,WWR,Recommended,Baseline,EUI_kBtu_per_sqft,TEDI_kBtu_per_sqft,CEDI_kBtu_per_sqft,EDI_kBtu_per_sqft,sensible_peak_cooling_W_per_sqft,sensible_peak_heating_W_per_sqft,time_of_peak_cooling,time_of_peak_heating,peak_heating_W_per_sqft,peak_cooling_W_per_sqft
RUN,,,,,,,,,,,,,,,
2,WINU,0.14,0.5,no,no,500.516211,54.386635,251.606950,305.993585,8.827118,0.500928,02-OCT-13:50,02-DEC-06:00,5.778187,12.520751
7,WINU,0.30,0.5,no,no,500.516211,54.386635,251.606950,305.993585,8.827118,0.500928,02-OCT-13:50,02-DEC-06:00,5.778187,12.520751
12,WINU,0.40,0.5,no,yes,500.516211,54.386635,251.606950,305.993585,8.827118,0.500928,02-OCT-13:50,02-DEC-06:00,5.778187,12.520751
17,WINU,0.50,0.5,no,no,500.516211,54.386635,251.606950,305.993585,8.827118,0.500928,02-OCT-13:50,02-DEC-06:00,5.778187,12.520751
22,WINU,0.80,0.5,no,no,500.516211,54.386635,251.606950,305.993585,8.827118,0.500928,02-OCT-13:50,02-DEC-06:00,5.778187,12.520751
10,WWR,0.40,0.1,no,no,507.116450,51.454628,251.859594,303.314222,7.683743,0.125843,02-OCT-13:40,02-DEC-06:00,4.974553,11.668987
11,WWR,0.40,0.3,no,no,502.072281,52.925274,250.851297,303.776571,8.253850,0.315003,02-OCT-13:40,02-DEC-06:00,5.374192,12.149128
12,WWR,0.40,0.5,no,yes,500.516211,54.386635,251.606950,305.993585,8.827118,0.500928,02-OCT-13:50,02-DEC-06:00,5.778187,12.520751
13,WWR,0.40,0.7,no,no,501.403510,55.980618,253.585861,309.566479,9.383575,0.644472,02-OCT-13:50,02-DEC-06:00,6.051573,13.365056


In [5]:
METRICS = [
    "EUI_kBtu_per_sqft",
    "TEDI_kBtu_per_sqft",
    "CEDI_kBtu_per_sqft",
    "EDI_kBtu_per_sqft",
]

nb_vars = df.Variable.unique().size


def plot_func(metric):
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(
        nrows=nb_vars // 3 or 1, ncols=min(nb_vars, 3), sharey=True
    )
    axes = axes.ravel()
    ylim = (df[metric].values.min(), df[metric].values.max())
    for i, (var, data) in enumerate(df.groupby("Variable")):
        data.drop_duplicates(inplace=True)
        if var == "WINU":
            data[var] = 1 / data[var]
        # plot the lineplot first
        ax = data.plot(
            x=var,  # plot var a x axis
            y=metric,  # plot metric a y axis
            color="#1f77b4",
            label=var,
            ax=axes[i],
        )
        # List of colors c for markers
        c = (
            data[[var, "Baseline"]]
            .set_index("Baseline")
            .apply(lambda x: "#b4d6e8" if x.name == "no" else "#1f77b4", axis=1)
            .tolist()
        )
        # Plot markers with scatter plot, second
        data.plot.scatter(
            x=var,
            y=metric,
            c=c,
            marker="o",
            s=50,
            edgecolors="#1f77b4",
            ax=axes[i],
        )
        # Plot the horizontal line of the baseline
        for h in data.loc[data["Baseline"] == "yes", metric]:
            ax.axhline(h, c="#eaeaea", zorder=-1)
        ax.set_ylabel(metric)  # set ylabel

In [6]:
%matplotlib inline

from ipywidgets import interact
import ipywidgets as widgets

interact(
    plot_func,
    metric=widgets.Dropdown(
        options=METRICS,
        index=0,
    ),
)

interactive(children=(Dropdown(description='metric', options=('EUI_kBtu_per_sqft', 'TEDI_kBtu_per_sqft', 'CEDI…

<function __main__.plot_func(metric)>

<!--NAVIGATION-->
< [Results From Parametric Analysis](01.03-Results-From-Parametric-Analysis.ipynb) | [Contents](Index.ipynb) | [Introduction to EnergyPlus Models Scripting](02.00-Introduction-to-EnergyPlus-Models-Scripting.ipynb) >